In [2]:
import numpy as np
import scipy.special # for the sigmoid activation function

import matplotlib.pyplot
%matplotlib inline

In [3]:
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.in_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.out_nodes = output_nodes
        self.learning_rate = learning_rate
        
        #  Initializing weights
        self.wih = np.random.normal(0.0, pow(self.hidden_nodes, -0.5), (self.hidden_nodes, self.in_nodes))
        self.who = np.random.normal(0.0, pow(self.out_nodes, -0.5), (self.out_nodes, self.hidden_nodes))
        
        # sigmoid activation function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        
    def train(self, inputs_list, targets_list):
        # make into 2D array
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # to adjust the weights between hidden and final layers
        output_errors = targets - final_outputs
        
        # to adjust the weights between the input and hidden layers
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # update the weights for the links between the hidden and output layers
        self.who += self.learning_rate * np.dot((output_errors * 
                    final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.learning_rate * np.dot((hidden_errors * 
                    hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
        
    def query(self, inputs_list):
        # make into 2D array
        inputs = np.array(inputs_list, ndmin=2).T
        
        # inputs * weights = the hidden layer node inputs
        hidden_inputs = np.dot(self.wih, inputs)
        # apply activation function to all the hidden node inputs
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # hidden layer outputs * weights = final layer inputs
        final_inputs = np.dot(self.who, hidden_outputs)
        # apply activation function to all final layer inputs
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [4]:
input_N = 784
hidden_N = 200
output_N = 10
learning_R = 0.1

n = NeuralNetwork(input_N, hidden_N, output_N, learning_R)

In [5]:
# Opening the CSV training file and converting it to a list
training_data_file = open("mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [6]:
# training the model
epochs = 2

for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]) / 255 * 0.99) + 0.01
    
        targets = np.zeros(output_N) + 0.01
    
        targets[int(all_values[0])] = 0.99
    
        n.train(inputs, targets)

In [7]:
# load the mnist test data CSV file into a list
test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [8]:
scorecard = []

for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    
    #print("Correct label is ", correct_label)
    
    scaled_input = (np.asfarray(all_values[1:]) / 255 * 0.99) + 0.01
    outputs = n.query(scaled_input)
    
    predicted_label = np.argmax(outputs)
    #print("Network's prediction ", predicted_label)
    #print()
    
    if (predicted_label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)

In [21]:
scorecard_array = np.asarray(scorecard)

avg = scorecard_array.sum() / scorecard_array.size
percentage = avg * 100

print("Performance = ", scorecard_array.sum() / scorecard_array.size, "or ", '%.2f' % percentage, '%')

Performance =  0.9674 or  96.74 %
